In [ ]:
using CSV, StatsBase, Statistics, DataFrames, UMAP, RCall
using Distributed, RMP, Random, Distributions
using MultivariateStats, MultipleTesting
using Dates: now
using LinearAlgebra: I, Diagonal, diag, det, qr, Symmetric

In [ ]:
@rlibrary ggplot2
@rlibrary extrafont

In [ ]:
R"""
# Used later for MCD computation

library(robustbase)

# Customize ggplot appearance

library(ggplot2)
library(extrafont)


# Load extra fonts
ttf_import(paths = "/tmp/.fonts/")
loadfonts()

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), 
                     legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
scale_fill_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)
"""

In [ ]:
# Number of points in control dataset
NR = 4500
# Number of points in other datasets
N = 1500
# Number of dimensions in each dataset
D = 100
# Percentage of datasets contaminated with outliers
pOutliers = 1/3
# Scaling of the transformation for positive controls
posScaling = 1.0;
# Number of dimensions kept for UMAP distance computation
dimUMAP = 10

## Dataset 1 - Reference R
We assume our data of interest to follow a multivariate normal distribution: In a morphological profiling, components are to some extent *independent (by removing correlated morphological features) and* normally distributed (by using a log-transformation).

In [ ]:
Random.seed!(1);

### Data center

In [ ]:
# The reference is centered on 0
µ = zeros(D);

### Data covariance

In [ ]:
# Diagonal: variances follow a Gamma distribution of shape and scale parameters equal to 1 and 2
# Rationale: Some variability in scales with some high values, and no negative values
# NB: Beta distribution could be used instead of Gamma distribution if long-tail is not needed
# NB: Effects are smoothed by the orthogonal transformation anyway
distrib = Gamma(1,2)
sigma_diag = rand(distrib, D);

In [ ]:
ggplot(DataFrame(x = sigma_diag)) + geom_histogram(aes(:x))

In [ ]:
# Now we transform this space by multiplying by a random orthogonal matrix
s = rand(D,D)
Q, R = qr(s);

In [ ]:
# NB: becomes really slow, do not try with D > 500
∑ = Q' * Diagonal(sigma_diag) * Q;

In [ ]:
# Check that the matrix is symmetrical (up to machine error)
@assert all([∑[i,j] ≈ ∑[j,i] for i in 1:D for j in 1:D if j>i])
# Make it perfectly symmetrical
[∑[i,j] = ∑[j,i] for i in 1:D for j in 1:D if j>i]
# Sylvester's criterion of positive semidefinite matrices
@assert all([det(∑[1:size,1:size]) > 0 for size in 1:D])

In [ ]:
ggplot(DataFrame(x = diag(∑))) + geom_histogram(aes(:x)) 

### Data center

In [ ]:
# The reference still is centered (center µ)
# The outliers are not centered on 0 anymore
distrib = Normal(0, 1)
µOutliers = rand(distrib, D)

### Data covariance

In [ ]:
# The covariance of the outliers is similar but independent of the reference points
distrib = Gamma(1,2)
sigma_diagOutliers = rand(distrib, D);

In [ ]:
# Now we transform this space by multiplying by a random orthogonal matrix
s = rand(D,D)
Q, R = qr(s);

In [ ]:
# NB: becomes really slow, do not try with D > 500
∑Outliers = Q' * Diagonal(sigma_diagOutliers) * Q;

In [ ]:
# Check that the matrix is symmetrical (up to machine error)
@assert all([∑Outliers[i,j] ≈ ∑Outliers[j,i] for i in 1:D for j in 1:D if j>i])
# Make it perfectly symmetrical
[∑Outliers[i,j] = ∑Outliers[j,i] for i in 1:D for j in 1:D if j>i]
# Sylvester's criterion of positive semidefinite matrices
@assert all([det(∑Outliers[1:size,1:size]) > 0 for size in 1:D])

In [ ]:
ggplot(DataFrame(x = diag(∑Outliers))) + geom_histogram(aes(:x)) 

### Output reference dataset

In [ ]:
# `100*pOutliers`% of the data will follow a multivariate normal distribution with the parameters
# we generated previously
distrib = MvNormal(µ, ∑);
distribOutliers = MvNormal(µOutliers, ∑Outliers);

In [ ]:
matR = DataFrame(hcat(rand(distrib, Int(round(NR*(1-pOutliers)))),
                      rand(distribOutliers, Int(round(NR*pOutliers))))')

## Dataset 2 - Positive control (shifted) P

In [ ]:
Random.seed!(2);

### Data center

In [ ]:
# The reference is not centered on 0 anymore
distrib = Normal(0, 0.5*posScaling)
µmod = rand(distrib, D)

### Output shifted dataset

In [ ]:
# The data will follow a multivariate normal distribution with the parameters
# we generated previously
distrib = MvNormal(µmod, ∑);

In [ ]:
matP = DataFrame(hcat(rand(distrib, Int(round(N*(1-pOutliers)))),
                      rand(distribOutliers, Int(round(N*pOutliers))))')

## Define distances of interest

In [ ]:
""" Compute the Mahalanobis Distance to center (MDC)
    in a dataset 'data' for a given perturbation of indices 'iPert' 
    compared to a reference of indices 'iRef'."""
function MDC(data, iPert, iRef)
    setPert = Matrix(data[iPert,:])
    setRef = Matrix(data[iRef,:])

    mdCenter = dropdims(mean(setRef, dims = 1), dims = 1)
    mdCov = cov(setRef)

    pertCenter = dropdims(mean(setPert, dims = 1), dims = 1)
    
    MD = mahalanobis(pertCenter, mdCenter, mdCov)
    
    return(MD)
end

In [ ]:
""" Compute the median Mahalanobis Distance (MD)
    in a dataset 'data' for a given perturbation of indices 'iPert' 
    compared to a reference of indices 'iRef'."""
function MD(data, iPert, iRef)
    setPert = data[iPert,:]
    setRef = Matrix(data[iRef,:])

    mdCenter = dropdims(mean(setRef, dims = 1), dims = 1)
    mdCov = cov(setRef)
    
    MD = median(map(x -> mahalanobis(x, mdCenter, mdCov), eachrow(setPert)))
    return(MD)
end

In [ ]:
""" Compute the median Robust Mahalanobis Distance (RMD)
    in a dataset 'data' for a given perturbation of indices 'iPert' 
    compared to a reference of indices 'iRef'.
    See https://e-archivo.uc3m.es/bitstream/handle/10016/24613/ws201710.pdf """
function RMD(data, iPert, iRef)
    setPert = data[iPert,:]
    setRef = data[iRef,:] 

    # Ensure that we have enough points to compute distance
    if ((size(setPert)[1] < 2*size(data, 2))|(size(setRef)[1] < 2*size(data, 2)))
        return(missing)
    end
    # NB: having less points than twice the number of dimensions leads to singularity
    
    # Compute Minimum Covariance Determinant and corresponding Robust Mahalanobis Distance
    @rput setRef

    R"""
    set.seed(3895)
    mcd <- covMcd(setRef)
    mcdCenter <- mcd$center
    mcdCov <- mcd$cov
    """
    @rget mcdCenter
    @rget mcdCov
    
    RMD = median(map(x -> mahalanobis(x, mcdCenter, mcdCov), eachrow(setPert)))
    return(RMD)
end

In [ ]:
""" Compute the Robust Hellinger Distance (RHD)
    in a dataset `data` for a given perturbation of indices `iPert` 
    compared to a reference of indices `iRef`."""
function RHD(data, iPert, iRef)
    setPert = data[iPert,:]
    setRef = data[iRef,:] 

    # Ensure that we have enough points to compute distance
    if ((size(setPert)[1] < 2*size(data, 2))|(size(setRef)[1] < 2*size(data, 2)))
        return(missing)
    end
    # NB: having less points than twice the number of dimensions leads to singularity
    
    # Compute Minimum Covariance Determinant and corresponding Robust Hellinger Distance
    @rput setRef
    @rput setPert

    R"""
    set.seed(3895)
    mcd1 <- covMcd(setRef)
    mcdCenter1 <- mcd1$center
    mcdCov1 <- mcd1$cov
    
    # We set the seed twice to always
    # find the same estimators given
    # the same sample
    set.seed(3895)
    mcd2 <- covMcd(setPert)
    mcdCenter2 <- mcd2$center
    mcdCov2 <- mcd2$cov
    """
    @rget mcdCenter1
    @rget mcdCov1
    @rget mcdCenter2
    @rget mcdCov2
    
    RHD = hellinger(mcdCenter1, mcdCov1, mcdCenter2, mcdCov2)
    return(RHD)
end

## Compare raw distances

In [ ]:
df = vcat(matR, matP)
# Remember how these records were generated
origDataset = vcat(repeat(["Reference"], size(matR, 1)),
                   repeat(["Positive control"], size(matP, 1)));

In [ ]:
# Actual observed MD to center
allMDCraw = map(x -> MDC(df, origDataset.==x,
                origDataset.=="Reference"), unique(origDataset))

In [ ]:
allMDraw = map(x -> MD(df, origDataset.==x,
                       origDataset.=="Reference"), unique(origDataset))

In [ ]:
allRMDraw = map(x -> RMD(df, origDataset.==x, 
                origDataset.=="Reference"), unique(origDataset))

In [ ]:
allRHDraw = map(x -> RHD(df, origDataset.==x, origDataset.=="Reference"),
                unique(origDataset))

## Compare PCA distances

In [ ]:
Random.seed!(3895)
modelPCA = fit(PCA, convert(Matrix, df)'; pratio = 0.9)
dimPCA = min(outdim(modelPCA), 50)
pcaND = MultivariateStats.transform(modelPCA, convert(Matrix, df)')
pcaND = convert(DataFrame, pcaND')
# Scale by importance of each principal component
pcaND = DataFrame(principalvars(modelPCA) .* eachcol(pcaND))[:,1:dimPCA]
names!(pcaND, Symbol.(string.("PC", 1:dimPCA)))

pcaND[:Condition] = origDataset;

In [ ]:
gp = ggplot(pcaND, aes(:PC1, :PC2)) + geom_point(aes(color = :Condition), alpha = 0.3)
print(gp)

In [ ]:
allMDCpca = map(x -> MDC(pcaND[:,1:dimPCA], pcaND.Condition.==x,
             pcaND.Condition.=="Reference"), unique(pcaND.Condition))

In [ ]:
allMDpca = map(x -> MD(pcaND[:,1:dimPCA], pcaND.Condition.==x, 
                       pcaND.Condition.=="Reference"), unique(pcaND.Condition))

In [ ]:
allRMDpca = map(x -> RMD(pcaND[:,1:dimPCA], pcaND.Condition.==x, 
                pcaND.Condition.=="Reference"), unique(pcaND.Condition))

In [ ]:
allRHDpca = map(x -> RHD(pcaND[:,1:dimUMAP], pcaND.Condition.==x, pcaND.Condition.=="Reference"),
                unique(pcaND.Condition))

## Compare UMAP distances

In [ ]:
# Visualization
Random.seed!(3895)
umND = umap(convert(Matrix, df)', 2; min_dist = 1, spread = 10, n_epochs = 200)
umND = convert(DataFrame, umND')
names!(umND, [:UMAP1, :UMAP2])

umND[:Condition] = origDataset

In [ ]:
gp = ggplot(umND, aes(:UMAP1, :UMAP2)) + geom_point(aes(color = :Condition), alpha = 0.3)
print(gp)

In [ ]:
# Re-run UMAP with more dimensions (to preserve more of the total information)
Random.seed!(3895)
umND = umap(convert(Matrix, df)', dimUMAP; min_dist = 1, spread = 10, n_epochs = 200)
umND = convert(DataFrame, umND')
names!(umND, Symbol.(string.("UMAP", 1:dimUMAP)))

umND[:Condition] = origDataset

In [ ]:
allMDC = map(x -> MDC(umND[:,1:dimUMAP], umND.Condition.==x, umND.Condition.=="Reference"), unique(umND.Condition))

In [ ]:
allMD = map(x -> MD(umND[:,1:dimUMAP], umND.Condition.==x, umND.Condition.=="Reference"), unique(umND.Condition))

In [ ]:
allRMD = map(x -> RMD(umND[:,1:dimUMAP], umND.Condition.==x, umND.Condition.=="Reference"), unique(umND.Condition))

In [ ]:
allRHD = map(x -> RHD(umND[:,1:dimUMAP], umND.Condition.==x, umND.Condition.=="Reference"), unique(umND.Condition))